<a href="https://colab.research.google.com/github/Roopana/TwitterAdAnalysis/blob/master/SentenceEmbeddings_superbowl_ads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U sentence-transformers

In [ ]:
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer('bert-base-nli-mean-tokens')


100%|██████████| 405M/405M [00:24<00:00, 16.7MB/s]


In [ ]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

stop = stopwords.words('english')
stop.append('superbowl')
stop.append('super')
stop.append('bowl')
stop.append('commercial')
stop.append('ad')
stop.append('commercials')
stop.append('ads')
print(len(stop))

def removeMentions(text):

    textBeforeMention = text.partition("@")[0]
    textAfterMention = text.partition("@")[2]
    textAfterMention =  re.sub(r':', '', textAfterMention) #cadillac join the 31k
    tHandle = textAfterMention.partition(" ")[0].lower() #cadillac    
    text = textBeforeMention+ " " + textAfterMention  
    return text

def cleanTweet(strinp):
    strinp = re.sub(r'RT', "", strinp) # Remove RT
    strinp = strinp.lower()
    
    stop_removed_list = [word for word in strinp.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_list])    
    text = re.sub('https?://[A-Za-z0-9./]+', ' ', stop_removed) # Remove URLs
    text = removeMentions(text)
    text = re.sub('[^\x00-\x7F]+', ' ', text) # Remove non-ASCII chars.
    
    # remove punctuations except '-'
    punctuation = ['(', ')', '[',']','?', ':', ':', ',', '.', '!', '/', '"', "'", '@', '#', '&', '-', '_']
    text = "".join((char for char in text if char not in punctuation))
    text = re.sub('[^a-zA-Z]', ' ', text) # remove all other than alphabet chars 

#     text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters     
    stop_removed_l = [word for word in text.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_l]) 
    return stop_removed

print(cleanTweet("RT @cadillacabc: Joinrt the 31K james_bond") )

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
186
cadillacabc joinrt k jamesbond


In [ ]:
import pandas as pd

# corpus = ['A man is eating food.',
#           'A man is eating a piece of bread.',
#           'The girl is carrying a baby.',
#           'A man is riding a horse.',
#           'A woman is playing violin.',
#           'Two men pushed carts through the woods.',
#           'A man is riding a white horse on an enclosed ground.',
#           'A monkey is playing drums.',
#           'A cheetah is running behind its prey.'
#           ]
df_1 = pd.read_csv("./mann_ann_sb.csv", index_col= None)

df_2 = pd.read_csv("./2_man_ann_sb.csv",  index_col= None)
df_2 = df_2.dropna(subset = ['tweet_text'])

df_3 = pd.read_csv("./3_man_ann_sb_full_1.csv", index_col = None)
df_3 = df_3.dropna(subset = ['tweet_text'])

df = df_1.append(df_2).append(df_3) # using batch 1 and batch 2 for training

df['text_clean'] = df['tweet_text'].apply(lambda x: cleanTweet(x))
df['ad_manual_adjusted'] = df['ad_manual_adjusted'].apply(lambda x: x.lower())
print(df.shape)

tweet_embeddings = embedder.encode(df['text_clean'], convert_to_tensor=True)


(7500, 20)


In [ ]:
# Query sentences:
ad_annotations = pd.read_csv('./SB_ad_annotations.csv')
ad_words = ad_annotations['Key Terms  Round 1']
print(ad_words.shape)
ad_embeddings = embedder.encode(ad_words, convert_to_tensor=True)
# queries = ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']


(75,)


Finding Semantic similarity between two sentences

In [ ]:

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 3
i =0
for query_embedding in tweet_embeddings:
    # query_embedding = embedder.encode(query, convert_to_tensor=True)
    if(i==100):
      break
    scores = util.pytorch_cos_sim(query_embedding, ad_embeddings)[0]

    results = zip(range(len(scores)), scores)
    results = sorted(results, key=lambda x: x[1], reverse=True)

    print("\n======================\n")
    print("Query:", df['text_clean'].iloc[i:i+1])
    print("\nTop 5 most similar sentences in corpus:")
    i = i+1
    for idx, score in results[0:closest_n]:
        print(ad_words[idx], "(Score: %.4f)" % (score))

Using Sentence Embeddings from Sentence Transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

In [ ]:
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    # print("Embedding:", embedding)
    print(embedding.shape)

Sentence: This framework generates embeddings for each input sentence
(768,)
Sentence: Sentences are passed as a list of string.
(768,)
Sentence: The quick brown fox jumps over the lazy dog.
(768,)


In [ ]:
# Use BERT for mapping tokens to embeddings
from sentence_transformers import models, losses

word_embedding_model = models.Transformer('bert-base-uncased')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
# model.fit(train_objectives=[(train_dataloader, train_loss)],
#           evaluator=evaluator,
#           epochs=num_epochs,
#           evaluation_steps=1000,
#           warmup_steps=warmup_steps,
#           output_path=model_save_path)